# Day 22

[Day 22 description](https://adventofcode.com/2018/day/22)

The most tricky part of this challenge was to search a path in a multi-state labyrinth. I found a solution which was not optimal, but fast enough to finish in minutes.

The main idea is to create a cave large enough to contain the whole explorative area. Then, a modified Dijkstra will keep track of both the position and the tool. If I reach a previous point, with the same tool but in less time, I start exploring again. Some time could be saved by memoizing all possible paths, but this kind of optimization was not necessary.

I don't have a perfect solution, meaning that I don't have a stopping condition. I just run the algorithm for enough time that no further optimizations are found for a long time (1200 steps are just enough!)

In [1]:
import numpy as np

In [2]:
with open('AOC2018_22_input.txt') as f:
    raws = [x.strip() for x in f.readlines()]

In [3]:
def to_int(xs): return [int(x) for x in xs]

In [4]:
def display_cave(cave):
    char_map = {0: '.', 1: '=', 2: '|'}
    res = ""
    x, y = cave.shape
    for i in range(y):
        row = "".join(char_map[x] for x in cave.T[i])
        res += row + "\n"
    return res

In [5]:
depth, (tx, ty) = int(raws[0][7:]), to_int(raws[1][8:].split(","))
#depth, (tx, ty) = 510, (10, 10)
w = tx + 200
h = ty + 200
gi_x = 16807
gi_y = 48271
el_m = 20183

In [6]:
def compute_gi(x, y, gi_cave):
    if x == 0 and y == 0:
        gi = 0
    elif x == tx and y == ty:
        gi = 0
    elif y == 0:
        gi = x * gi_x
    elif x == 0:
        gi = y * gi_y
    else:
        gi = el(gi_cave[x-1, y])*el(gi_cave[x, y-1]) % el_m
    return gi

In [7]:
def el(gi):
    return (gi + depth) % el_m

In [8]:
gi_cave = np.zeros((w, h), dtype=int)
for x in range(w):
    for y in range(h):
        gi_cave[x,y] = compute_gi(x, y, gi_cave)
cave = el(gi_cave) % 3
cave[:tx+1,:ty+1].sum()

11575

In [9]:
tools = {
    0: {'climb', 'torch'},
    1: {'climb','neither'},
    2: {'torch','neither'}
}

def move_cost(source, target, old_tool):
    sp0, sp1 = source
    tp0, tp1 = target
    source_tile = cave[sp0, sp1]
    target_tile = cave[tp0, tp1]
    common_tools = tools[source_tile].intersection(tools[target_tile])
    if old_tool in common_tools:
        return (1, old_tool)
    else:
        new_tool = next(t for t in common_tools)
        return (8, new_tool)

def move(p, d):
    m = {'U': [0,-1], 'L': [-1,0], 'D': [0,1], 'R': [1,0]}
    px, py = p
    dx, dy = m[d]
    return (px + dx, py + dy)

def next_steps(p, t):
    steps = []
    if p[0] > 0:
        steps.append('L')
    if p[1] > 0:
        steps.append('U')
    if p[0] < w - 1:
        steps.append('R')
    if p[1] < h - 1:
        steps.append('D')
    return [(move(p, d), t) for d in steps]

In [10]:
exploration = {
    ((0,0), 'torch'): 0
}
new_points = dict()
for new_p, prev_tool in next_steps((0,0), 'torch'):
    new_time, new_tool = move_cost((0,0), new_p, prev_tool)
    new_points[(new_p, new_tool)] = new_time
exploration.update(new_points)
prev_points = dict(new_points)

In [11]:
i = 0
while True:
    i += 1
    if i > 1200:
        break
    new_points = dict()
    for (old_p, old_tool), old_time in prev_points.items():
        for new_p, prev_tool in next_steps(old_p, old_tool):
            if new_p == (tx, ty):
                pass
                #print("target reached!")
            new_time, new_tool = move_cost(old_p, new_p, prev_tool)
            if (new_p, new_tool) in exploration:
                prev_time = exploration[(new_p, new_tool)]
                if prev_time <= old_time + new_time:
                    pass
                else:
                    if new_p == (tx, ty):
                        print("target optimized at %s steps!" % i)
                        print(new_p, new_tool, old_time + new_time)
                    new_points[(new_p, new_tool)] = old_time + new_time
                    exploration[(new_p, new_tool)] = old_time + new_time
            else:
                new_points[(new_p, new_tool)] = old_time + new_time
                exploration[(new_p, new_tool)] = old_time + new_time
    exploration.update(new_points)
    prev_points = dict(new_points)

target optimized at 793 steps!
(14, 778) climb 2103
target optimized at 795 steps!
(14, 778) torch 2105
target optimized at 795 steps!
(14, 778) climb 2049
target optimized at 797 steps!
(14, 778) torch 2030
target optimized at 797 steps!
(14, 778) climb 1988
target optimized at 799 steps!
(14, 778) torch 1990
target optimized at 799 steps!
(14, 778) climb 1948
target optimized at 801 steps!
(14, 778) torch 1943
target optimized at 801 steps!
(14, 778) climb 1901
target optimized at 803 steps!
(14, 778) torch 1903
target optimized at 803 steps!
(14, 778) climb 1861
target optimized at 805 steps!
(14, 778) climb 1821
target optimized at 805 steps!
(14, 778) torch 1863
target optimized at 807 steps!
(14, 778) torch 1823
target optimized at 807 steps!
(14, 778) climb 1781
target optimized at 809 steps!
(14, 778) climb 1748
target optimized at 809 steps!
(14, 778) torch 1783
target optimized at 811 steps!
(14, 778) torch 1750
target optimized at 811 steps!
(14, 778) climb 1715
target optim

In [12]:
enpoints = [(p, tool, t) for (p, tool), t in exploration.items() if p == (tx, ty)]

In [13]:
endtime_with_torch = [t for (p, tool), t in exploration.items() if p == (tx, ty) and tool == 'torch'][0]
endtime_with_no_torch = min([t for (p, tool), t in exploration.items() if p == (tx, ty) and tool != 'torch']) + 7
min(endtime_with_torch, endtime_with_no_torch)

1068